In [1]:
import pandas as pd
import numpy as np
from numpy import asarray

from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing import sequence
from unidecode import unidecode
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Dropout
from keras.layers import GaussianNoise
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.models import load_model
from keras.preprocessing import sequence


from os.path import isfile
import os
import pickle
import sys
from nltk.tokenize import word_tokenize


from unidecode import unidecode
# from tf_data import TF_Data

/Users/evgenia/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data_file="Combined_News_DJIA.csv"

In [3]:
df = pd.read_csv(data_file)

In [4]:
len(df)

1989

In [5]:
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [6]:
df=df[:100]

In [7]:
len(df)

100

In [8]:

def str_join(df, sep, *cols):
    from functools import reduce
    return reduce(lambda x, y: x.astype(str).str.cat(y.astype(str), sep=sep), 
                [df[col] for col in cols])
        

In [9]:
df['headlines'] = str_join(df, 'Top1', 'Top2', 'Top3', 'Top4', 'Top5',"Top6",'Top7','Top8','Top9','Top10','Top11','Top12','Top13','Top14','Top15','Top16','Top17','Top18','Top19','Top20','Top21','Top22','Top23','Top24','Top25')

In [10]:
df = df[["Date","Label", "headlines"]]

In [11]:
dataset=df["headlines"].tolist()

In [12]:
print len(dataset)

100


In [13]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk

# nltk.download()
stopWords = set(stopwords.words('english'))
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
df["headlines"]=df.apply(lambda row: tokenizer.tokenize(row['headlines'] ), axis=1)


df["headlines"].head()

df["headlines"]=df["headlines"].apply(lambda x: [item for item in x if item not in stopWords])



k=df['headlines'].apply(lambda x: [item for item in x if item not in stopWords])
print k


0     [b, BREAKING, Musharraf, impeached, Top1b, Rus...
1     [b, Bush, puts, foot, Georgian, conflict, Top1...
2     [b, Russia, ends, Georgia, operation, Top1b, I...
3     [b, When, president, ordered, attack, Tskhinva...
4     [b, War, South, Osetia, 89, pictures, made, Ru...
5     [b, Russia, U, S, Poland, Missile, Deal, Won, ...
6     [b, Little, girl, ugly, Top1b, Pakistan, Musha...
7     [b, The, US, missile, defence, system, magic, ...
8     [b, The, Power, Islam, The, Human, Rights, Cou...
9     [b, Chinese, may, killed, 140, Tibetans, week,...
10    [b, Superclass, new, book, past, several, deca...
11    [b, Secret, prison, Diego, Garcia, confirmed, ...
12    [b, 60, Children, Among, Dead, US, Airstrike, ...
13    [b, London, Olympics, Tacky, 2012, handover, s...
14    [b, Western, Media, Mislead, Public, Georgia, ...
15    [b, Michael, Lee, White, resident, Houston, bo...
16    [b, The, Dutch, recalled, spy, Iran, citing, U...
17    [b, What, REALLY, happening, Russia, U, S,

In [14]:
dataset=k.tolist()

In [27]:
max_index=0
for a in k:
#     print len(word_tokenize(a))
    if len(a)>max_index :
        max_index=len(a)
        
print max_index

296


In [15]:
ds=[]
for i in k:
    ds.append(str(i))
# print dataset

In [16]:
dataset=ds

In [29]:
max_index=0
for a in dataset:
#     print len(word_tokenize(a))
    if len(a)>max_index :
        max_index=len(a)
        
print max_index

2834


In [30]:
len(dataset[0])

2718

In [31]:
MAX_NB_WORDS=300
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(dataset)
data = np.zeros((len(dataset),MAX_NB_WORDS),dtype="int32")

In [33]:
ln=[]
for i, sentences in enumerate(dataset):
#     for j, sent in enumerate(sentences):
    if i< MAX_NB_WORDS:
        wordTokens = text_to_word_sequence(sentences)
        k=0 
        ln.append(len(wordTokens))
        for _, word in enumerate(wordTokens):
#             print word
            data[i,k] = tokenizer.word_index[word]
            k=k+1
            

In [34]:
data

array([[   9,  556, 1019, ...,    0,    0,    0],
       [   9,   71, 1029, ...,    0,    0,    0],
       [   9,    4,  664, ...,    0,    0,    0],
       ...,
       [   9, 1009, 1635, ...,    0,    0,    0],
       [   9,  289, 2909, ...,    0,    0,    0],
       [   9,    7,  942, ...,    0,    0,    0]], dtype=int32)

In [35]:
word_index=tokenizer.word_index
len(word_index)

6636

In [36]:
embeddings_index = {}
GLOVE_DIR="./"
GLOVE_FILE="glove.6B.300d copy.txt"
f = open(os.path.join(GLOVE_DIR, GLOVE_FILE))
for line in f:
    values = line.split()
    if len(values)>0:
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
f.close()

IOError: [Errno 2] No such file or directory: './glove.6B.300d copy.txt'

In [59]:
EMBEDDING_DIM=300
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                embedding_matrix[i] = embedding_vector

In [60]:
labels=df["Label"].tolist()
labels=np.array(labels)
# validation_split=0.2

train_size = int(len(labels) * 0.80)
test_size = len(labels) - train_size

X_train=data[0:train_size]
y_train=labels[0:train_size]
X_test=data[train_size:len(labels)]
y_test=labels[train_size:len(labels)]

In [61]:
NUM_NEURONS=50
DROP=0.2
L_R=0.1
BATCH_SIZE=30
embedding_layer = Embedding(input_dim = len(word_index) + 1,
                            output_dim=EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_NB_WORDS)
#                             mask_zero=True

earlyStopping=EarlyStopping(monitor='acc', patience=2, verbose=1, mode='auto') 
model = Sequential()
model.add(embedding_layer)

model.add(GaussianNoise(0.1))
model.add(LSTM(NUM_NEURONS))

model.add(Dropout(DROP))
# model.add(Dense(1, activation='sigmoid'),activity_regularizer=l2(0.0001))
model.add(Dense(1, activation='sigmoid'))
optimizer = Adam(lr=L_R)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

print(model.summary())

history=model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=BATCH_SIZE, callbacks=[earlyStopping])

fd = open('accuracy_DJ__.csv','a')
CsvRow=["Model","NUM_EPOCHS","NUM_NEURONS","Dropout","Adam_lr","BATCH_SIZE","train_acc","test_acc"]
scores_test = model.evaluate(X_test, y_test, verbose=0)
scores_train=model.evaluate(X_train, y_train, verbose=0)

print scores_train[1]*100
print scores_test[1]*100

CsvRowTrain = ["L","EarlyStopping",str(NUM_NEURONS),str(DROP),str(L_R),str(BATCH_SIZE),str(scores_train[1]*100) ,str(scores_test[1]*100)]
print CsvRowTrain
fd.write(", ".join(CsvRowTrain) + "\n")
fd.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 700, 300)          16971600  
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 700, 300)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                70200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 17,041,851
Trainable params: 17,041,851
Non-trainable params: 0
_________________________________________________________________
None
Train on 1591 samples, validate on 398 samples
Epoch 1/100
1591/1591 [==============================] - 150s - loss: 0.8433 -

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()